# A.P. Moller Maersk Case Study - DS_ML Coding Challenge Dataset
 

**Table of contents**<a id='toc0_'></a>    
- [Preprocessing and Feature Engineering](#toc1_)    
  - [Reading all Sheets in Excel File and displaying them](#toc1_1_)    
  - [Intializing the training and testing dataframes](#toc1_2_)    
  - [About the Data](#toc1_3_)    
  - [Changing the data types of the columns](#toc1_4_)    
- [Exploratory Data Analysis on the Data](#toc2_)    
  - [Training Data](#toc2_1_)    
    - [Missing Values](#toc2_1_1_)    
    - [Category Distribution](#toc2_1_2_)    
    - [Date wise Distribution](#toc2_1_3_)    
    - [Data Visualization on the Training Data (categorical features)](#toc2_1_4_)    
    - [Extracting Quantity of the Different Products Combinations](#toc2_1_5_)    
      - [Unique Product Combinations over Product Type , Manufacturer , Area Code , Sourcing Channel , Product Size , Product Type (Powder vs. Liquid)](#toc2_1_5_1_)    
      - [All combination line plot over months for Quantity and Sourcing Cost](#toc2_1_5_2_)    
    - [Adding the Quantity of the Different Products Combinations with respect to `Date` and `Sourcing Cost` to the Training Data](#toc2_1_6_)    
      - [Saving the Training Data to a CSV file](#toc2_1_6_1_)    
  - [Test Data](#toc2_2_)    
    - [Data Visualization on the Testing Data (categorical features)](#toc2_2_1_)    
- [Forecasting the Sourcing Cost](#toc3_)    
  - [Train Test Preparation](#toc3_1_)    
  - [Models](#toc3_2_)    
    - [LSTM Model](#toc3_2_1_)    
    - [XGBoost Regressor](#toc3_2_2_)    
    - [Random Forest Regressor](#toc3_2_3_)    
- [Forecasting the Quantity](#toc4_)    
  - [Train Test Split](#toc4_1_)    
  - [Models](#toc4_2_)    
    - [LSTM Model](#toc4_2_1_)    
    - [XGBoost Regressor](#toc4_2_2_)    
    - [Random Forest Regressor](#toc4_2_3_)    
- [Approach to the Problem Statement](#toc5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Preprocessing and Feature Engineering](#toc0_)

## <a id='toc1_1_'></a>[Reading all Sheets in Excel File and displaying them](#toc0_)

In [1]:
import pandas as pd

# Path to the Excel file
file_path = "DS_ML Coding Challenge Dataset.xlsx"

# Load all sheets into a dictionary of DataFrames
all_sheets = pd.read_excel(file_path, sheet_name=None)

# Access each sheet using its name
for sheet_name, df in all_sheets.items():
    print(f"Data from sheet: {sheet_name}")
    print(df.head())  # Display the first few rows of the DataFrame
    print("\n")


Data from sheet: Instructions
   Unnamed: 0 Unnamed: 1                                         Unnamed: 2
0         1.0   Dataset:  Every row represents the sourcing of one unit ...
1         NaN        NaN  A unique product combination comprises of attr...
2         NaN        NaN  Since each row represents 1 unit of sourcing; ...
3         NaN        NaN  July 20 to May 21 is your training set and Jun...
4         NaN        NaN  June 2021 has only a single value for each com...


Data from sheet: Training Dataset
  ProductType Manufacturer Area Code Sourcing Channel Product Size  \
0        NTM3           X1       A28        WHOLESALE        Large   
1        NTM2           X1        A9           DIRECT        Large   
2        NTM3           X2       A20           DIRECT        Large   
3        NTM3           X1       A18        WHOLESALE        Small   
4        NTM2           X1       A28           DIRECT        Large   

  Product Type Month of Sourcing  Sourcing Cost  
0      

In [2]:
all_sheets.keys()

dict_keys(['Instructions', 'Training Dataset', 'Test Dataset'])

## <a id='toc1_2_'></a>[Intializing the training and testing dataframes](#toc0_)

In [3]:
train_df = all_sheets['Training Dataset']
test_df = all_sheets['Test Dataset']

In [4]:
train_df.head()

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost
0,NTM3,X1,A28,WHOLESALE,Large,Powder,2021-05-01,10.158
1,NTM2,X1,A9,DIRECT,Large,Powder,2020-10-01,134.281
2,NTM3,X2,A20,DIRECT,Large,Powder,2020-12-01,12.456
3,NTM3,X1,A18,WHOLESALE,Small,Powder,2021-02-01,107.220
4,NTM2,X1,A28,DIRECT,Large,Liquid,2020-11-01,197.763


In [5]:
test_df.head()

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost
0,NTM1,X1,A1,DIRECT,Small,Powder,2021-06-21,103.68
1,NTM1,X1,A10,DIRECT,Large,Powder,2021-06-21,155.75
2,NTM1,X1,A10,ECOM,Large,Powder,2021-06-21,143.02
3,NTM1,X1,A11,DIRECT,Large,Powder,2021-06-21,139.39
4,NTM1,X1,A2,DIRECT,Large,Powder,2021-06-21,169.42


## <a id='toc1_3_'></a>[About the Data](#toc0_)

In [6]:
print("Train dataset Info")
print(train_df.info())

print("\n")

print("Test dataset Info")
print(test_df.info())

Train dataset Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550176 entries, 0 to 550175
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ProductType        550176 non-null  object        
 1   Manufacturer       550176 non-null  object        
 2   Area Code          550176 non-null  object        
 3   Sourcing Channel   550176 non-null  object        
 4   Product Size       550176 non-null  object        
 5   Product Type       550176 non-null  object        
 6   Month of Sourcing  550176 non-null  datetime64[ns]
 7   Sourcing Cost      550176 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 33.6+ MB
None


Test dataset Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0

## <a id='toc1_4_'></a>[Changing the data types of the columns](#toc0_)

In [7]:
import pandas as pd

test_df['Month of Sourcing'] = pd.to_datetime(test_df['Month of Sourcing'], errors='coerce')

test_df['Year'] = test_df['Month of Sourcing'].dt.year
test_df['Month'] = test_df['Month of Sourcing'].dt.month


In [8]:
import pandas as pd
import calendar


train_df['Month of Sourcing'] = pd.to_datetime(train_df['Month of Sourcing'], errors='coerce')

train_df['Year'] = train_df['Month of Sourcing'].dt.year
train_df['Month'] = train_df['Month of Sourcing'].dt.month

month_to_name = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "October", 11: "November", 12: "December"
}



# <a id='toc2_'></a>[Exploratory Data Analysis on the Data](#toc0_)

## <a id='toc2_1_'></a>[Training Data](#toc0_)

In [9]:
print("Train dataset Info")
print(train_df.info())

Train dataset Info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550176 entries, 0 to 550175
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ProductType        550176 non-null  object        
 1   Manufacturer       550176 non-null  object        
 2   Area Code          550176 non-null  object        
 3   Sourcing Channel   550176 non-null  object        
 4   Product Size       550176 non-null  object        
 5   Product Type       550176 non-null  object        
 6   Month of Sourcing  550176 non-null  datetime64[ns]
 7   Sourcing Cost      550176 non-null  float64       
 8   Year               550176 non-null  int64         
 9   Month              550176 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 42.0+ MB
None


In [10]:
train_df.head()

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost,Year,Month
0,NTM3,X1,A28,WHOLESALE,Large,Powder,2021-05-01,10.158,2021,5
1,NTM2,X1,A9,DIRECT,Large,Powder,2020-10-01,134.281,2020,10
2,NTM3,X2,A20,DIRECT,Large,Powder,2020-12-01,12.456,2020,12
3,NTM3,X1,A18,WHOLESALE,Small,Powder,2021-02-01,107.220,2021,2
4,NTM2,X1,A28,DIRECT,Large,Liquid,2020-11-01,197.763,2020,11


### <a id='toc2_1_1_'></a>[Missing Values](#toc0_)

In [11]:
# Check for missing values
print(df.isnull().sum())

ProductType          0
Manufacturer         0
Area Code            0
Sourcing Channel     0
Product Size         0
Product Type         0
Month of Sourcing    0
Sourcing Cost        0
Year                 0
Month                0
dtype: int64


### <a id='toc2_1_2_'></a>[Category Distribution](#toc0_)

In [12]:
train_df["ProductType"].value_counts()

NTM2    236726
NTM1    194923
NTM3    118527
Name: ProductType, dtype: int64

In [13]:
train_df["Manufacturer"].value_counts()

X1    419857
X2    120695
X3      9624
Name: Manufacturer, dtype: int64

In [14]:
train_df["Area Code"].value_counts()

A28    41925
A7     36723
A3     33247
A11    31111
A8     28772
A44    26490
A5     24252
A10    22970
A25    20422
A31    18379
A29    18105
A16    15938
A12    14547
A40    13820
A21    13374
A43    13234
A2     13145
A1     12676
A6     12399
A4     11326
A24    10725
A18    10154
A9     10107
A22     9624
A14     9424
A35     8877
A45     8188
A13     7548
A42     6470
A33     5769
A15     5496
A32     5408
A36     4843
A34     4249
A19     3839
A38     3065
A26     2360
A30     2357
A20     2126
A46     1732
A39     1702
A37     1432
A17     1139
A23      569
A41      118
Name: Area Code, dtype: int64

### <a id='toc2_1_3_'></a>[Date wise Distribution](#toc0_)

In [15]:
data_date = train_df["Month of Sourcing"].value_counts().reset_index()
data_date.columns = ["Month of Sourcing", "Count"]

In [16]:
data_date

,Month of Sourcing,Count
0,2020-11-01,60446
1,2021-03-01,56643
2,2021-05-01,53172
3,2020-12-01,52752
4,2021-04-01,52438
5,2021-01-01,50844
6,2021-02-01,50562
7,2020-10-01,46215
8,2020-09-01,43995
9,2020-07-01,42469


In [17]:
import pandas as pd
import calendar

data_date['Month of Sourcing'] = pd.to_datetime(data_date['Month of Sourcing'], errors='coerce')

# Extracting year and month as new columns
data_date['Year'] = data_date['Month of Sourcing'].dt.year
data_date['Month'] = data_date['Month of Sourcing'].dt.month

# Defining a dictionary mapping month numbers to names
month_to_name = {
    1: "January", 2: "February", 3: "March", 4: "April",
    5: "May", 6: "June", 7: "July", 8: "August",
    9: "September", 10: "October", 11: "November", 12: "December"
}

data_date['Month Name'] = data_date['Month'].map(month_to_name)


In [18]:
data_date

,Month of Sourcing,Count,Year,Month,Month Name
0,2020-11-01,60446,2020,11,November
1,2021-03-01,56643,2021,3,March
2,2021-05-01,53172,2021,5,May
3,2020-12-01,52752,2020,12,December
4,2021-04-01,52438,2021,4,April
5,2021-01-01,50844,2021,1,January
6,2021-02-01,50562,2021,2,February
7,2020-10-01,46215,2020,10,October
8,2020-09-01,43995,2020,9,September
9,2020-07-01,42469,2020,7,July


### <a id='toc2_1_4_'></a>[Data Visualization on the Training Data (categorical features)](#toc0_)

In [19]:
import pandas as pd
import plotly.express as px  # Using Plotly Express for interactive plots

# If using a Jupyter Notebook, use the following to enable interactive visuals in the notebook
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)


In [20]:
import plotly.express as px

categorical_columns = train_df.select_dtypes(include=['object' , "datetime64[ns]"]).columns

# Dictionary to store plot objects
plot_objects = {}

for column in categorical_columns:
    # Prepare the data
    data = train_df[column].value_counts().reset_index()
    data.columns = [column, 'Counts']

    # Create the plot
    fig = px.bar(data, 
                 x=column, 
                 y='Counts', 
                 text='Counts',  # Adding text to bars
                 labels={column: 'Category', 'Counts': 'Counts'}, 
                 title=f'Value Counts of {column}')
    

    # Customizing text position and styling
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    
    # Customizing layout and axes
    fig.update_layout(
        xaxis_title=column,
        yaxis_title="Counts",
        title={
            'text': f'Value Counts for {column}',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_tickangle=-45 , # Angle the x-axis labels for better readability
        height=600,  # Set the height of the plot
        width=1000,  # Set the width of the plot
    )
    
    # Save plot object to dictionary
    plot_objects[column] = fig


# For example, to display the plot for a specific column:
# column_name = 'Column_Name'
# plot_objects[column_name].show()


In [21]:
plot_objects.keys()

dict_keys(['ProductType', 'Manufacturer', 'Area Code', 'Sourcing Channel', 'Product Size', 'Product Type', 'Month of Sourcing'])

In [22]:
plot_objects['ProductType'].show()  # Display the plot for ProductType

<div class="alert alert-block alert-info">
    <strong>Product Type Distribution:</strong> The distribution of product types reveals that <b>NTM2</b> is the most prevalent type, followed by <b>NTM1</b> and <b>NTM3</b>. This indicates a potential demand pattern or production focus within the market.
</div>



In [23]:
plot_objects['Manufacturer'].show()  # Display the plot for Manufacturer

<div class="alert alert-block alert-info">
    <strong>Manufacturer Distribution:</strong> Manufacturer <b>X1</b> dominates the market with the highest count, suggesting it holds a significant share compared to <b>X2</b> and <b>X3</b>. Understanding the market dynamics between these manufacturers can provide strategic insights for stakeholders.
</div>

In [24]:
plot_objects['Area Code'].show()  # Display the plot for Area Code

<div class="alert alert-block alert-info">
    <strong>Area Code Distribution:</strong> Area code <b>A28</b> has the highest count, indicating a potential concentration of customers or demand in that area. Analyzing the demographics and preferences of customers in this area could help tailor marketing or distribution strategies.
</div>

In [25]:
plot_objects['Sourcing Channel'].show()  # Display the plot for Sourcing Channel


<div class="alert alert-block alert-info">
    <strong>Sourcing Channel Distribution:</strong> <b>Direct</b> sourcing channel leads by a substantial margin, indicating a preference for direct transactions. Understanding the factors driving this preference can help optimize channel strategies and enhance customer satisfaction.
</div>



In [26]:
plot_objects['Product Size'].show()  # Display the plot for Product Size

<div class="alert alert-block alert-info">
    <strong>Product Size Distribution:</strong> <b>Large</b>-sized products dominate the market, followed by <b>small</b>-sized products. This suggests a preference for larger product sizes among consumers or perhaps a strategic emphasis on producing larger variants.
</div>



In [27]:
plot_objects['Product Type'].show()  # Display the plot for Product Type

<div class="alert alert-block alert-info">
    <strong>Product Type (Powder vs. Liquid) Distribution:</strong> <b>Powder</b> products significantly out<b>liquid products</b>, indicating a higher demand or production volume for powder-based items. Understanding the factors driving this preference can inform product development and marketing strategies.
</div>


In [28]:
plot_objects['Month of Sourcing'].show()  # Display the plot for Month of Sourcing

<div class="alert alert-block alert-info">
    <strong>Month of Sourcing Distribution:</strong> The distribution of sourcing across months highlights fluctuations in sourcing activity over time. Peaks and troughs in counts can be analyzed in conjunction with external factors such as seasonal demand, economic conditions, or marketing campaigns to identify patterns and optimize sourcing strategies.
</div>

In [29]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550176 entries, 0 to 550175
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ProductType        550176 non-null  object        
 1   Manufacturer       550176 non-null  object        
 2   Area Code          550176 non-null  object        
 3   Sourcing Channel   550176 non-null  object        
 4   Product Size       550176 non-null  object        
 5   Product Type       550176 non-null  object        
 6   Month of Sourcing  550176 non-null  datetime64[ns]
 7   Sourcing Cost      550176 non-null  float64       
 8   Year               550176 non-null  int64         
 9   Month              550176 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 42.0+ MB


### <a id='toc2_1_5_'></a>[Extracting Quantity of the Different Products Combinations](#toc0_)

#### <a id='toc2_1_5_1_'></a>[Unique Product Combinations over Product Type , Manufacturer , Area Code , Sourcing Channel , Product Size , Product Type (Powder vs. Liquid)](#toc0_)

In [30]:
grouping_columns_unique = ['ProductType', 'Manufacturer', 'Area Code', 'Sourcing Channel', 
                    'Product Size', 'Product Type']


import pandas as pd

unique_combinations = train_df[grouping_columns_unique].drop_duplicates()
unique_combinations

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type
0,NTM3,X1,A28,WHOLESALE,Large,Powder
1,NTM2,X1,A9,DIRECT,Large,Powder
2,NTM3,X2,A20,DIRECT,Large,Powder
3,NTM3,X1,A18,WHOLESALE,Small,Powder
4,NTM2,X1,A28,DIRECT,Large,Liquid
...,...,...,...,...,...,...
138809,NTM2,X1,A23,RETAIL,Large,Powder
138815,NTM1,X1,A3,DIRECT,Small,Powder
138819,NTM2,X1,A26,DIRECT,Small,Powder
138864,NTM1,X2,A42,DIRECT,Large,Powder


<div class="alert alert-block alert-warning">
    <strong>Insights:</strong> The unique product combinations provide a comprehensive view of the market landscape, highlighting the diversity of offerings across different categories. Analyzing the distribution of these combinations can reveal patterns in consumer preferences, market trends, and strategic opportunities for product development and marketing.<br>
    Also the test data has exact same unique product combinations which are equal to <b>96</b> as the training data mean the model can be trained on the training data and tested on the test data.
</div>

#### <a id='toc2_1_5_2_'></a>[All combination line plot over months for Quantity and Sourcing Cost](#toc0_)

In [31]:
data2 = pd.read_csv("train_df_quantify.csv") 


data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1021 entries, 0 to 1020
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ProductType        1021 non-null   object 
 1   Manufacturer       1021 non-null   object 
 2   Area Code          1021 non-null   object 
 3   Sourcing Channel   1021 non-null   object 
 4   Product Size       1021 non-null   object 
 5   Product Type       1021 non-null   object 
 6   Month of Sourcing  1021 non-null   object 
 7   Sourcing Cost      1021 non-null   float64
 8   Year               1021 non-null   int64  
 9   Month              1021 non-null   int64  
 10  Quantity           1021 non-null   int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 87.9+ KB


In [32]:
data2.head()

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost,Year,Month,Quantity
0,NTM3,X1,A28,WHOLESALE,Large,Powder,2021-05-01,10.158,2021,5,79
1,NTM2,X1,A9,DIRECT,Large,Powder,2020-10-01,134.281,2020,10,135
2,NTM3,X2,A20,DIRECT,Large,Powder,2020-12-01,12.456,2020,12,197
3,NTM3,X1,A18,WHOLESALE,Small,Powder,2021-02-01,107.220,2021,2,768
4,NTM2,X1,A28,DIRECT,Large,Liquid,2020-11-01,197.763,2020,11,271


In [33]:
df = data2

In [34]:
import pandas as pd
import calendar
import plotly.graph_objs as go

grouping_columns_unique = ['ProductType', 'Manufacturer', 'Area Code', 'Sourcing Channel', 'Product Size', 'Product Type']

# Group by unique combinations and aggregate mean Sourcing Cost and sum Quantity
grouped = df.groupby(grouping_columns_unique + ['Year', 'Month']).agg({'Sourcing Cost':'mean', 'Quantity':'sum'}).reset_index()

# Merge with unique_combinations to retain only those combinations
merged = pd.merge(grouped, unique_combinations, on=grouping_columns_unique, how='inner')

# Initialize a dictionary to store monthly data for each unique product combination
product_data = {}

# Iterate over unique combinations
for index, row in merged.iterrows():
    product_label = f"{row['ProductType']} - {row['Manufacturer']} - {row['Area Code']} - {row['Sourcing Channel']} - {row['Product Size']} - {row['Product Type']}"
    if product_label not in product_data:
        product_data[product_label] = []
    
    monthly_data = {
        'Month': calendar.month_name[row['Month']],
        'Year': str(row['Year']),
        'Sourcing Cost': row['Sourcing Cost'],
        'Quantity': row['Quantity']
    }
    
    product_data[product_label].append(monthly_data)

# Create a Plotly figure to hold the plots
fig = go.Figure()

# Iterate over each unique product combination
for product_label, monthly_data in product_data.items():
    # Extract data for each metric (Sourcing Cost and Quantity)
    months = [data['Month'] for data in monthly_data]
    sourcing_cost = [data['Sourcing Cost'] for data in monthly_data]
    quantity = [data['Quantity'] for data in monthly_data]
    
    # Add traces for Sourcing Cost and Quantity
    fig.add_trace(go.Scatter(x=months, y=sourcing_cost, mode='lines+markers', name=f'{product_label} - Sourcing Cost', text=sourcing_cost, textposition='top center'))
    fig.add_trace(go.Scatter(x=months, y=quantity, mode='lines+markers', name=f'{product_label} - Quantity', text=quantity, textposition='top center'))

# Update layout for better readability
fig.update_layout(title='Change in Sourcing Cost and Quantity Over Months for Unique Product Combinations',
                  xaxis_title='Month',
                  yaxis_title='Value',
                  legend_title='Product Combinations',
                  hovermode='closest',
                  
                  )

# Show the plot
fig.show()

fig.write_html('sourcing_cost_quantity_plot.html', full_html=False, include_plotlyjs='cdn')



<div class="alert alert-block alert-info">
    Interactive plot for visualizing the unique product combinations and their corresponding quantities and Sourcing Cost. The plot allows users to explore the distribution of product combinations across different categories and identify patterns or trends in the data.
    
</div>


<div class="alert alert-block alert-info">
    <b>Note : Double click on the combination to see the individual plot in legend on right side. Then single click for comparison two or more combinations.</b>
</div>


In [35]:
len(product_data)

96

### <a id='toc2_1_6_'></a>[Adding the Quantity of the Different Products Combinations with respect to `Date` and `Sourcing Cost` to the Training Data](#toc0_)

In [36]:
train_df_quantify = train_df.copy()

In [37]:
import pandas as pd

# Columns to group by
grouping_columns = ['ProductType', 'Manufacturer', 'Area Code', 'Sourcing Channel', 
                    'Product Size', 'Product Type', 'Month of Sourcing']

# Create a groupby object and count the number of occurrences in each group
train_df_quantify['Quantity'] = train_df_quantify.groupby(grouping_columns)['Sourcing Cost'].transform('count')

train_df_quantify = train_df_quantify.drop_duplicates(subset=grouping_columns)



In [38]:
train_df_quantify.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1021 entries, 0 to 422767
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ProductType        1021 non-null   object        
 1   Manufacturer       1021 non-null   object        
 2   Area Code          1021 non-null   object        
 3   Sourcing Channel   1021 non-null   object        
 4   Product Size       1021 non-null   object        
 5   Product Type       1021 non-null   object        
 6   Month of Sourcing  1021 non-null   datetime64[ns]
 7   Sourcing Cost      1021 non-null   float64       
 8   Year               1021 non-null   int64         
 9   Month              1021 non-null   int64         
 10  Quantity           1021 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 95.7+ KB


In [39]:
train_df_quantify.head()

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost,Year,Month,Quantity
0,NTM3,X1,A28,WHOLESALE,Large,Powder,2021-05-01,10.158,2021,5,79
1,NTM2,X1,A9,DIRECT,Large,Powder,2020-10-01,134.281,2020,10,135
2,NTM3,X2,A20,DIRECT,Large,Powder,2020-12-01,12.456,2020,12,197
3,NTM3,X1,A18,WHOLESALE,Small,Powder,2021-02-01,107.220,2021,2,768
4,NTM2,X1,A28,DIRECT,Large,Liquid,2020-11-01,197.763,2020,11,271


In [40]:
train_df_quantify['Quantity'].sum()

550176

Total Quantity is equal to the sum of the quantity of the different product combinations. ( as the dataset has single entry as one unit)

In [41]:
train_df_quantify['Year'].value_counts()

2020    545
2021    476
Name: Year, dtype: int64

In [42]:
train_df_quantify['Month'].value_counts()

5     96
2     96
11    95
3     95
1     95
4     94
12    93
10    91
9     90
8     89
7     87
Name: Month, dtype: int64

In [43]:
sorted_df = train_df_quantify[['Month', 'Year']].value_counts().reset_index(name='Count').sort_values(by=['Year', 'Month'])


In [44]:
sorted_df

,Month,Year,Count
10,7,2020,87
9,8,2020,89
8,9,2020,90
7,10,2020,91
4,11,2020,95
6,12,2020,93
2,1,2021,95
0,2,2021,96
3,3,2021,95
5,4,2021,94


In [45]:
sorted_df_quantity = train_df_quantify.groupby(['Year', 'Month']).agg({'Quantity': 'sum'}).reset_index().sort_values(by=['Year', 'Month'])


In [46]:
sorted_df_quantity

,Year,Month,Quantity
0,2020,7,42469
1,2020,8,40640
2,2020,9,43995
3,2020,10,46215
4,2020,11,60446
5,2020,12,52752
6,2021,1,50844
7,2021,2,50562
8,2021,3,56643
9,2021,4,52438


#### <a id='toc2_1_6_1_'></a>[Saving the Training Data to a CSV file](#toc0_)

In [47]:
# train_df_quantify.to_csv('train_df_quantify.csv', index=False)

In [48]:
unique_combinations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 139034
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   ProductType       96 non-null     object
 1   Manufacturer      96 non-null     object
 2   Area Code         96 non-null     object
 3   Sourcing Channel  96 non-null     object
 4   Product Size      96 non-null     object
 5   Product Type      96 non-null     object
dtypes: object(6)
memory usage: 5.2+ KB


## <a id='toc2_2_'></a>[Test Data](#toc0_)

In [49]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ProductType        96 non-null     object        
 1   Manufacturer       96 non-null     object        
 2   Area Code          96 non-null     object        
 3   Sourcing Channel   96 non-null     object        
 4   Product Size       96 non-null     object        
 5   Product Type       96 non-null     object        
 6   Month of Sourcing  96 non-null     datetime64[ns]
 7   Sourcing Cost      96 non-null     float64       
 8   Year               96 non-null     int64         
 9   Month              96 non-null     int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 7.6+ KB


### <a id='toc2_2_1_'></a>[Data Visualization on the Testing Data (categorical features)](#toc0_)

In [50]:
categorical_columns = test_df.drop(columns=['Month of Sourcing']).select_dtypes(include=['object' , "datetime64[ns]"]).columns

for column in categorical_columns:
    # Prepare the data
    data = test_df[column].value_counts().reset_index()
    data.columns = [column, 'Counts']

    # Create the plot
    fig = px.bar(data, 
                 x=column, 
                 y='Counts', 
                 text='Counts',  # Adding text to bars
                 labels={column: 'Category', 'Counts': 'Counts'}, 
                 title=f'Value Counts of {column}')
    

    # Customizing text position and styling
    fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
    
    # Customizing layout and axes
    fig.update_layout(
        xaxis_title=column,
        yaxis_title="Counts",
        title={
            'text': f'Value Counts for {column}',
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        xaxis_tickangle=-45 , # Angle the x-axis labels for better readability
        height=600,  # Set the height of the plot
        width=1000,  # Set the width of the plot
    )
    
    fig.show()

# <a id='toc3_'></a>[Forecasting the Sourcing Cost](#toc0_)

In [51]:
train_df_quantify.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1021 entries, 0 to 422767
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ProductType        1021 non-null   object        
 1   Manufacturer       1021 non-null   object        
 2   Area Code          1021 non-null   object        
 3   Sourcing Channel   1021 non-null   object        
 4   Product Size       1021 non-null   object        
 5   Product Type       1021 non-null   object        
 6   Month of Sourcing  1021 non-null   datetime64[ns]
 7   Sourcing Cost      1021 non-null   float64       
 8   Year               1021 non-null   int64         
 9   Month              1021 non-null   int64         
 10  Quantity           1021 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 95.7+ KB


## <a id='toc3_1_'></a>[Train Test Preparation](#toc0_)

In [52]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# Load data
data = pd.read_csv("train_df_quantify.csv")  # Replace "your_data.csv" with your file path



# Label encoding for categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le


train_data = data.copy()

# Prepare sequences for LSTM
X_train = train_data.drop(columns=['Quantity', 'Month of Sourcing','Sourcing Cost'])
y_train  = train_data['Sourcing Cost']


In [53]:
# Prepare test data
test_data = test_df.copy()


# Label encoding for categorical variables
label_encoders = {}
for column in test_data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    test_data[column] = le.fit_transform(test_data[column])
    label_encoders[column] = le



X_test = test_data.drop(columns=['Month of Sourcing' ,'Sourcing Cost'])


# Reshape input data for LSTM
X_test_lstm = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))




In [54]:
X_test.shape

(96, 8)

## <a id='toc3_2_'></a>[Models](#toc0_)

### <a id='toc3_2_1_'></a>[LSTM Model](#toc0_)

In [55]:
X_train.shape

(1021, 8)

In [56]:
# Step 2: Define the LSTM model
model = Sequential()
model.add(LSTM(units=100, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Step 3: Train the model
# Reshape input data for LSTM
X_train_lstm = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))

# Fit the model
model.fit(X_train_lstm, y_train, epochs=50, batch_size=32)



Epoch 1/50
32/32 [==============================] - 1s 4ms/step - loss: 26769.8066
Epoch 2/50
32/32 [==============================] - 0s 4ms/step - loss: 25779.5898
Epoch 3/50
32/32 [==============================] - 0s 4ms/step - loss: 25738.2090
Epoch 4/50
32/32 [==============================] - 0s 4ms/step - loss: 25719.0254
Epoch 5/50
32/32 [==============================] - 0s 4ms/step - loss: 25724.1816
Epoch 6/50
32/32 [==============================] - 0s 4ms/step - loss: 25822.6680
Epoch 7/50
32/32 [==============================] - 0s 4ms/step - loss: 25719.0625
Epoch 8/50
32/32 [==============================] - 0s 4ms/step - loss: 25754.4629
Epoch 9/50
32/32 [==============================] - 0s 4ms/step - loss: 25669.1562
Epoch 10/50
32/32 [==============================] - 0s 4ms/step - loss: 25677.2617
Epoch 11/50
32/32 [==============================] - 0s 5ms/step - loss: 25621.0059
Epoch 12/50
32/32 [==============================] - 0s 4ms/step - loss: 25618.2344
E

In [57]:
test_df.head()

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost,Year,Month
0,NTM1,X1,A1,DIRECT,Small,Powder,2021-06-21,103.68,2021,6
1,NTM1,X1,A10,DIRECT,Large,Powder,2021-06-21,155.75,2021,6
2,NTM1,X1,A10,ECOM,Large,Powder,2021-06-21,143.02,2021,6
3,NTM1,X1,A11,DIRECT,Large,Powder,2021-06-21,139.39,2021,6
4,NTM1,X1,A2,DIRECT,Large,Powder,2021-06-21,169.42,2021,6


In [58]:
# Step 4: Forecast the quantity for June 2021


# Make predictions
predictions_lstm = model.predict(X_test_lstm)

# Display forecast for June 2021
print("Forecasted quantity for June 2021:")
print(predictions_lstm)





3/3 [==============================] - 0s 3ms/step
Forecasted quantity for June 2021:
[[113.1758  ]
 [134.86537 ]
 [157.62425 ]
 [137.67064 ]
 [141.0608  ]
 [234.9983  ]
 [132.72983 ]
 [135.42331 ]
 [137.22021 ]
 [136.63428 ]
 [138.15623 ]
 [146.69362 ]
 [147.9236  ]
 [149.21234 ]
 [152.67252 ]
 [153.96559 ]
 [153.27475 ]
 [151.4819  ]
 [151.23181 ]
 [154.58517 ]
 [153.89302 ]
 [155.13078 ]
 [154.52603 ]
 [ 85.54539 ]
 [ 55.984722]
 [ 73.125084]
 [ 66.12614 ]
 [ 86.423874]
 [ 90.52951 ]
 [ 95.47975 ]
 [ 97.87636 ]
 [ 99.68182 ]
 [ 98.46239 ]
 [100.83708 ]
 [ 89.71833 ]
 [161.26082 ]
 [177.44241 ]
 [167.83788 ]
 [164.6144  ]
 [151.52126 ]
 [270.79352 ]
 [170.16084 ]
 [172.70796 ]
 [172.2866  ]
 [443.07156 ]
 [149.71579 ]
 [152.38571 ]
 [151.8133  ]
 [145.97762 ]
 [147.80074 ]
 [143.37468 ]
 [145.17552 ]
 [193.54723 ]
 [186.74104 ]
 [158.96779 ]
 [158.88947 ]
 [159.02527 ]
 [159.11717 ]
 [154.40843 ]
 [159.2429  ]
 [151.1008  ]
 [161.01927 ]
 [161.09338 ]
 [162.27116 ]
 [161.49217 ]
 [16

In [59]:
# Extracting original sourcing cost values for June 2021 from the test data
original_sourcing_cost = test_df['Sourcing Cost'].values

# Creating DataFrame to display original and predicted sourcing cost values side by side
cost_comparison_df_lstm = pd.DataFrame({'Original Sourcing Cost': original_sourcing_cost, 'Predicted Sourcing Cost': predictions_lstm.flatten()})

# Display the DataFrame
print("Original and Predicted Sourcing Costs for June 2021:")
#Difference between original and predicted sourcing cost
cost_comparison_df_lstm['Difference'] = cost_comparison_df_lstm['Original Sourcing Cost'] - cost_comparison_df_lstm['Predicted Sourcing Cost']
cost_comparison_df_lstm

Original and Predicted Sourcing Costs for June 2021:


,Original Sourcing Cost,Predicted Sourcing Cost,Difference
0,103.68,113.175797,-9.495797
1,155.75,134.865372,20.884628
2,143.02,157.624252,-14.604252
3,139.39,137.670639,1.719361
4,169.42,141.060806,28.359194
...,...,...,...
91,89.57,99.364281,-9.794281
92,114.57,109.933800,4.636200
93,111.26,111.785751,-0.525751
94,32.32,43.267811,-10.947811


In [60]:
cost_comparison_df_lstm['Difference'].mean()

-27.485716911951698

### <a id='toc3_2_2_'></a>[XGBoost Regressor](#toc0_)

In [61]:
X_train.shape

(1021, 8)

In [62]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)



c:\Users\jaini\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\compat.py:93: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [63]:
#Predictions
y_pred = xgb_reg.predict(X_test)


In [64]:
# Extracting original sourcing cost values for June 2021 from the test data
original_sourcing_cost = test_df['Sourcing Cost'].values

# Creating a DataFrame to display original and predicted sourcing cost values side by side
cost_comparison_df_xgb = pd.DataFrame({'Original Sourcing Cost': original_sourcing_cost, 'Predicted Sourcing Cost': y_pred.flatten()})

# Display the DataFrame
print("Original and Predicted Sourcing Costs for June 2021:")
#Difference between original and predicted sourcing cost
cost_comparison_df_xgb['Difference'] = cost_comparison_df_xgb['Original Sourcing Cost'] - cost_comparison_df_xgb['Predicted Sourcing Cost']
print(cost_comparison_df_xgb)

Original and Predicted Sourcing Costs for June 2021:
    Original Sourcing Cost  Predicted Sourcing Cost  Difference
0                   103.68               116.454384  -12.774384
1                   155.75               151.400452    4.349548
2                   143.02               154.844589  -11.824589
3                   139.39               135.797363    3.592637
4                   169.42               162.202789    7.217211
..                     ...                      ...         ...
91                   89.57                66.181244   23.388756
92                  114.57               120.000061   -5.430061
93                  111.26               114.907295   -3.647295
94                   32.32                17.602777   14.717223
95                   40.73                68.420876  -27.690876

[96 rows x 3 columns]


In [65]:
cost_comparison_df_xgb['Difference'].mean()

-4.207570535739262

### <a id='toc3_2_3_'></a>[Random Forest Regressor](#toc0_)

In [66]:
from sklearn.ensemble import RandomForestRegressor

# Train Random Forest model
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)


RandomForestRegressor()

In [67]:
#Predictions
y_pred_rf = rf_reg.predict(X_test)

In [68]:
# Extracting original sourcing cost values for June 2021 from the test data
original_sourcing_cost = test_df['Sourcing Cost'].values

# Creating a DataFrame to display original and predicted sourcing cost values side by side
cost_comparison_df_rf = pd.DataFrame({'Original Sourcing Cost': original_sourcing_cost, 'Predicted Sourcing Cost': y_pred_rf.flatten()})

# Display the DataFrame
print("Original and Predicted Sourcing Costs for June 2021:")
#Difference between original and predicted sourcing cost
cost_comparison_df_rf['Difference'] = cost_comparison_df_rf['Original Sourcing Cost'] - cost_comparison_df_rf['Predicted Sourcing Cost']
cost_comparison_df_rf

Original and Predicted Sourcing Costs for June 2021:


,Original Sourcing Cost,Predicted Sourcing Cost,Difference
0,103.68,104.32052,-0.64052
1,155.75,153.90076,1.84924
2,143.02,150.95842,-7.93842
3,139.39,145.41162,-6.02162
4,169.42,160.18885,9.23115
...,...,...,...
91,89.57,65.63639,23.93361
92,114.57,123.27361,-8.70361
93,111.26,109.78551,1.47449
94,32.32,33.49756,-1.17756


In [69]:
cost_comparison_df_rf['Difference'].mean()

-7.709350312499983

In [70]:
# Evaluate model performance 
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(original_sourcing_cost, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 1096.7922305627067


# <a id='toc4_'></a>[Forecasting the Quantity](#toc0_)

In [71]:
train_df_quantify.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1021 entries, 0 to 422767
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ProductType        1021 non-null   object        
 1   Manufacturer       1021 non-null   object        
 2   Area Code          1021 non-null   object        
 3   Sourcing Channel   1021 non-null   object        
 4   Product Size       1021 non-null   object        
 5   Product Type       1021 non-null   object        
 6   Month of Sourcing  1021 non-null   datetime64[ns]
 7   Sourcing Cost      1021 non-null   float64       
 8   Year               1021 non-null   int64         
 9   Month              1021 non-null   int64         
 10  Quantity           1021 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 95.7+ KB


## <a id='toc4_1_'></a>[Train Test Split](#toc0_)

In [72]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

# Load data
data = pd.read_csv("train_df_quantify.csv")  



# Label encoding for categorical variables
label_encoders = {}
for column in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le



# Filter data for training (July 2020 to May 2021)
train_data = data.copy()

# Prepare sequences for LSTM
X_train = train_data.drop(columns=['Month of Sourcing','Quantity'])
y_train  = train_data['Quantity']

In [73]:
# Prepare test data
test_data = test_df.copy()


# Label encoding for categorical variables
label_encoders = {}
for column in test_data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    test_data[column] = le.fit_transform(test_data[column])
    label_encoders[column] = le


X_test = test_data.drop(columns=['Month of Sourcing'])


# Reshape input data for LSTM
X_test_lstm = X_test.values.reshape((X_test.shape[0], X_test.shape[1], 1))

In [74]:
X_test.head()

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Sourcing Cost,Year,Month
0,0,0,0,0,2,1,103.68,2021,6
1,0,0,1,0,1,1,155.75,2021,6
2,0,0,1,1,1,1,143.02,2021,6
3,0,0,2,0,1,1,139.39,2021,6
4,0,0,11,0,1,1,169.42,2021,6


## <a id='toc4_2_'></a>[Models](#toc0_)

### <a id='toc4_2_1_'></a>[LSTM Model](#toc0_)

In [75]:
X_train.shape

(1021, 9)

In [76]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(units=100, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
# Reshape input data for LSTM
X_train_lstm = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))

# Fit the model
model.fit(X_train_lstm, y_train, epochs=100, batch_size=32)



Epoch 1/100
32/32 [==============================] - 1s 4ms/step - loss: 350748.1250
Epoch 2/100
32/32 [==============================] - 0s 5ms/step - loss: 226602.5312
Epoch 3/100
32/32 [==============================] - 0s 4ms/step - loss: 223681.8281
Epoch 4/100
32/32 [==============================] - 0s 5ms/step - loss: 224398.6094
Epoch 5/100
32/32 [==============================] - 0s 4ms/step - loss: 223398.7344
Epoch 6/100
32/32 [==============================] - 0s 4ms/step - loss: 221834.7969
Epoch 7/100
32/32 [==============================] - 0s 4ms/step - loss: 223376.8594
Epoch 8/100
32/32 [==============================] - 0s 6ms/step - loss: 222088.0781
Epoch 9/100
32/32 [==============================] - 0s 5ms/step - loss: 220686.0000
Epoch 10/100
32/32 [==============================] - 0s 5ms/step - loss: 220296.2500
Epoch 11/100
32/32 [==============================] - 0s 5ms/step - loss: 219844.8281
Epoch 12/100
32/32 [==============================] - 0s 4ms/st

In [77]:
test_df.head()

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost,Year,Month
0,NTM1,X1,A1,DIRECT,Small,Powder,2021-06-21,103.68,2021,6
1,NTM1,X1,A10,DIRECT,Large,Powder,2021-06-21,155.75,2021,6
2,NTM1,X1,A10,ECOM,Large,Powder,2021-06-21,143.02,2021,6
3,NTM1,X1,A11,DIRECT,Large,Powder,2021-06-21,139.39,2021,6
4,NTM1,X1,A2,DIRECT,Large,Powder,2021-06-21,169.42,2021,6


In [78]:
# Step 4: Forecast the quantity for June 2021

# Make predictions
predictions_lstm = model.predict(X_test_lstm)

# Display forecast for July 2021
print("Forecasted quantity for June 2021:")
print(predictions_lstm)





3/3 [==============================] - 0s 3ms/step
Forecasted quantity for June 2021:
[[ 974.8107  ]
 [ 661.12775 ]
 [ 708.8583  ]
 [ 729.6732  ]
 [ 364.08652 ]
 [ 374.5717  ]
 [ 223.75891 ]
 [ 223.03369 ]
 [ 605.63007 ]
 [ 699.53625 ]
 [ 675.6361  ]
 [ 488.39523 ]
 [ 468.16257 ]
 [ 517.1162  ]
 [ 482.92578 ]
 [ 459.7905  ]
 [ 497.12122 ]
 [ 483.11734 ]
 [ 481.99802 ]
 [ 451.12033 ]
 [ 451.73895 ]
 [ 393.0187  ]
 [ 395.94272 ]
 [1452.3019  ]
 [ 871.1982  ]
 [ 162.4043  ]
 [ 291.43753 ]
 [ 727.5924  ]
 [ 891.5113  ]
 [ 210.41986 ]
 [ 507.50333 ]
 [ 591.5658  ]
 [ 342.9808  ]
 [ 276.2936  ]
 [ 177.28787 ]
 [ 878.55206 ]
 [ 852.0938  ]
 [ 892.5221  ]
 [ 883.1294  ]
 [ 396.146   ]
 [ 464.04053 ]
 [ 780.66003 ]
 [ 338.31705 ]
 [ 340.8507  ]
 [ 340.64352 ]
 [ 352.93576 ]
 [ 366.7739  ]
 [ 693.7385  ]
 [ 853.41785 ]
 [ 769.66345 ]
 [ 759.84875 ]
 [ 873.66895 ]
 [ 186.34879 ]
 [ 193.85443 ]
 [ 798.34515 ]
 [ 853.36414 ]
 [ 781.65393 ]
 [ 758.756   ]
 [ 204.8772  ]
 [ 666.16785 ]
 [  59.047966]

In [79]:
test_df_lstm = test_df.copy()
# Create a DataFrame to display original and predicted quantity values side by side
quantity_comparison_df_lstm = pd.DataFrame({'Predicted quantity': predictions_lstm.flatten()})
# Display the DataFrame
print("Predicted quantity for June 2021:")
quantity_comparison_df_lstm


Predicted quantity for June 2021:


,Predicted quantity
0,974.810730
1,661.127747
2,708.858276
3,729.673218
4,364.086517
...,...
91,680.255310
92,566.196655
93,567.162109
94,597.447876


In [80]:
test_df_lstm['Predicted quantity'] = predictions_lstm.flatten().astype(int)
test_df_lstm

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost,Year,Month,Predicted quantity
0,NTM1,X1,A1,DIRECT,Small,Powder,2021-06-21,103.68,2021,6,974
1,NTM1,X1,A10,DIRECT,Large,Powder,2021-06-21,155.75,2021,6,661
2,NTM1,X1,A10,ECOM,Large,Powder,2021-06-21,143.02,2021,6,708
3,NTM1,X1,A11,DIRECT,Large,Powder,2021-06-21,139.39,2021,6,729
4,NTM1,X1,A2,DIRECT,Large,Powder,2021-06-21,169.42,2021,6,364
...,...,...,...,...,...,...,...,...,...,...,...
91,NTM3,X1,A44,DIRECT,Small,Liquid,2021-06-21,89.57,2021,6,680
92,NTM3,X1,A8,DIRECT,Large,Powder,2021-06-21,114.57,2021,6,566
93,NTM3,X1,A8,DIRECT,Small,Powder,2021-06-21,111.26,2021,6,567
94,NTM3,X2,A20,DIRECT,Large,Powder,2021-06-21,32.32,2021,6,597


### <a id='toc4_2_2_'></a>[XGBoost Regressor](#toc0_)

In [81]:
X_train.shape

(1021, 9)

In [82]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

xgb_reg = xgb.XGBRegressor()
xgb_reg.fit(X_train, y_train)



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [83]:
#Predictions
y_pred_xgb = xgb_reg.predict(X_test)


In [84]:
test_df_xgb = test_df.copy()

# Creating a DataFrame to display original and predicted quantity values side by side
quantity_df_xgb = pd.DataFrame({'Predicted Quantity': y_pred.flatten()})

# Display the DataFrame
print("Predicted Quantity for June 2021:")
quantity_df_xgb

Predicted Quantity for June 2021:


,Predicted Quantity
0,116.454384
1,151.400452
2,154.844589
3,135.797363
4,162.202789
...,...
91,66.181244
92,120.000061
93,114.907295
94,17.602777


In [85]:
test_df_xgb['Predicted quantity'] = predictions_lstm.flatten().astype(int)
test_df_xgb

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost,Year,Month,Predicted quantity
0,NTM1,X1,A1,DIRECT,Small,Powder,2021-06-21,103.68,2021,6,974
1,NTM1,X1,A10,DIRECT,Large,Powder,2021-06-21,155.75,2021,6,661
2,NTM1,X1,A10,ECOM,Large,Powder,2021-06-21,143.02,2021,6,708
3,NTM1,X1,A11,DIRECT,Large,Powder,2021-06-21,139.39,2021,6,729
4,NTM1,X1,A2,DIRECT,Large,Powder,2021-06-21,169.42,2021,6,364
...,...,...,...,...,...,...,...,...,...,...,...
91,NTM3,X1,A44,DIRECT,Small,Liquid,2021-06-21,89.57,2021,6,680
92,NTM3,X1,A8,DIRECT,Large,Powder,2021-06-21,114.57,2021,6,566
93,NTM3,X1,A8,DIRECT,Small,Powder,2021-06-21,111.26,2021,6,567
94,NTM3,X2,A20,DIRECT,Large,Powder,2021-06-21,32.32,2021,6,597


### <a id='toc4_2_3_'></a>[Random Forest Regressor](#toc0_)

In [86]:
from sklearn.ensemble import RandomForestRegressor

# Train Random Forest model
rf_reg = RandomForestRegressor()
rf_reg.fit(X_train, y_train)


RandomForestRegressor()

In [87]:
#Predictions
y_pred_rf = rf_reg.predict(X_test)

In [88]:
test_df_rf = test_df.copy()

# Creating a DataFrame to display original and predicted quantity values side by side
quantity_df_rf = pd.DataFrame({'Predicted Quantity': y_pred_rf.flatten()})

# Display the DataFrame
print("Predicted Quantity for July 2021:")
#Difference between original and predicted quantity
quantity_df_rf

Predicted Quantity for July 2021:


,Predicted Quantity
0,1041.12
1,503.53
2,290.26
3,896.91
4,422.03
...,...
91,698.83
92,485.11
93,184.58
94,283.69


In [89]:
test_df_rf['Predicted quantity'] = predictions_lstm.flatten().astype(int)
test_df_rf

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost,Year,Month,Predicted quantity
0,NTM1,X1,A1,DIRECT,Small,Powder,2021-06-21,103.68,2021,6,974
1,NTM1,X1,A10,DIRECT,Large,Powder,2021-06-21,155.75,2021,6,661
2,NTM1,X1,A10,ECOM,Large,Powder,2021-06-21,143.02,2021,6,708
3,NTM1,X1,A11,DIRECT,Large,Powder,2021-06-21,139.39,2021,6,729
4,NTM1,X1,A2,DIRECT,Large,Powder,2021-06-21,169.42,2021,6,364
...,...,...,...,...,...,...,...,...,...,...,...
91,NTM3,X1,A44,DIRECT,Small,Liquid,2021-06-21,89.57,2021,6,680
92,NTM3,X1,A8,DIRECT,Large,Powder,2021-06-21,114.57,2021,6,566
93,NTM3,X1,A8,DIRECT,Small,Powder,2021-06-21,111.26,2021,6,567
94,NTM3,X2,A20,DIRECT,Large,Powder,2021-06-21,32.32,2021,6,597


In [90]:
cost_comparison_df_rf['Difference'].mean()

-7.709350312499983

# <a id='toc5_'></a>[Approach to the Problem Statement](#toc0_)




<div class="alert alert-block alert-warning">

The problem statement involves forecasting the <b>Sourcing Cost</b> and <b>Quantity</b> for different product combinations based on categorical features such as <b>Product Type</b>, <b>Manufacturer</b>, <b>Area Code</b>, <b>Sourcing Channel</b>, <b>Product Size</b>, and <b>Product Type (Powder vs. Liquid)</b>. The dataset contains historical data on the sourcing cost and quantity for various product combinations over different months.

So I first tired to find if I can make a quantity column for the given dataset , their are total 96 unique product combinations in the dataset. So I extracted the quantity of the different product combinations and added it to the training data. Then I visualized the data and then I forecasted the <b>Sourcing Cost</b> and <b>Quantity</b> using different models like LSTM, XGBoost Regressor, and Random Forest Regressor.

This approach is currently not working as the data is less if more data provided then the models will be able to predict the values more accurately.
Detailed explanation is provided below.
</div>

<div class="alert alert-block alert-success">

<div class="problem-statement">

<h3>The Problem Statement:</h3>

<p>The problem statement involves forecasting the Sourcing Cost and Quantity for different product combinations based on categorical features such as Product Type, Manufacturer, Area Code, Sourcing Channel, Product Size, and Product Type (Powder vs. Liquid). The dataset contains historical data on the sourcing cost and quantity for various product combinations over different months.</p>
</div>

<div class="approach">

<h3>Approach:</h3>

<ol>
	<li><strong>Data Preprocessing and Feature Engineering:</strong>
		<ul>
			<li>Read the dataset from the given Excel file containing multiple sheets.</li>
			<li>Use the 'Training Dataset' and 'Test Dataset' sheets for further analysis.</li>
			<li>Convert the 'Month of Sourcing' column to datetime format and create new 'Year' and 'Month' columns.</li>
			<li>Extract the quantity of different product combinations and add it to the training data.</li>
		</ul>
	</li>
	<li><strong>Exploratory Data Analysis (EDA):</strong>
		<ul>
			<li>Perform EDA on the training dataset to understand the data distribution, missing values, and category distribution.</li>
			<li>Create visualizations for categorical features using bar charts.</li>
		</ul>
	</li>
	<li><strong>Data Visualization on Training Data (categorical features):</strong>
		<ul>
			<li>Create bar charts for each categorical feature in the training dataset, showing the value counts of each category.</li>
			<li>This helps in understanding the distribution of categories in the dataset.</li>
		</ul>
	</li>
	<li><strong>Forecasting the Sourcing Cost:</strong>
		<ul>
			<li>Use LSTM, XGBoost Regressor, and Random Forest Regressor models for forecasting the sourcing cost.</li>
			<li>Train the models on the training dataset and make predictions on the test dataset.</li>
			<li>Compare the original and predicted sourcing costs using a DataFrame and calculate the difference between them.</li>
		</ul>
	</li>
	<li><strong>Forecasting the Quantity:</strong>
		<ul>
			<li>Use the LSTM model for forecasting the quantity.</li>
			<li>Train the model on the training dataset and make predictions on the test dataset.</li>
			<li>Display the predicted quantity for June 2021.</li>
		</ul>
	</li>
</ol>
</div>

<div class="improvements">

<h3>Improvements:</h3>

<ol>
	<li>Gather more historical data on the sourcing cost and quantity for various product combinations to improve the forecasting accuracy.</li>
	<li>Create additional features such as moving averages, trend, and seasonality components for the sourcing cost and quantity.</li>
	<li>Tune the hyperparameters of the LSTM, XGBoost Regressor, and Random Forest Regressor models to achieve better forecasting results.</li>
	<li>Create ensemble models by combining the predictions of the LSTM, XGBoost Regressor, and Random Forest Regressor models for more accurate and robust forecasting results.</li>
</ol>
</div>

<div class="conclusion">

<h3>Conclusion:</h3>

<p>The given approach involves proper data preprocessing, exploratory data analysis, and forecasting of sourcing cost and quantity using various machine learning models. The outputs and visualizations provided help in understanding the data distribution and the performance of the models used for forecasting. The approach can be further improved by gathering more data, creating additional features, tuning the models, and using ensemble models.</p>
</div>
</div>

# Author

<div class="alert alert-block alert-info">
    <p><strong>Name:</strong> Jainil Patel</p>
    <p><strong>College:</strong> Symbiosis Institute of Technology, Pune</p>
    <p><strong>Branch:</strong> Artificial Intelligence and Machine Learning (AI ML)</p>
    <p><strong>Year:</strong> 3rd Year</p>
    <p><strong>PRN:</strong> 21070126039</p>
    <p><strong>Email:</strong> <a href="mailto:jainil24680@gmail.com">jainil24680@gmail.com</a></p>
    <p><strong>LinkedIn:</strong> <a href="https://www.linkedin.com/in/jainil-patel-2857ab202/">Jainil Patel</a></p>
    <p><strong>GitHub:</strong> <a href="https://github.com/astro215">astro215</a></p>
    <p><strong>Resume:</strong> <a href="https://flowcv.com/resume/q3t7eurpt6">Jainil Patel's Resume</a></p>
</div>


# Acknowledgements
Thanks to the company for providing the DS_ML Coding Challenge and the opportunity to work on this interesting problem.